In [1]:
from new_model import * 

In [2]:
seed = 100
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.5**0.194

basic = 4

# multistepLR

## cifar10

In [8]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
최초 node의 갯수 : (tensor(1792., device='cuda:1'), tensor(1792., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:1'), tensor(1564., device='cuda:1'))
Epoch 10: progress time is 440.43 sec
Validation loss: 0.0206, Validation accuracy: 0.9952
Epoch 20: progress time is 877.6 sec
Validation loss: 0.0185, Validation accuracy: 0.9944
Epoch 30: progress time is 1315.38 sec
Validation loss: 0.0336, Validation accuracy: 0.9896
Epoch 40: progress time is 1754.01 sec
Validation loss: 0.0368, Validation accuracy: 0.9864
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:1'), tensor(1368., device='cuda:1'))
Epoch 50: progress time is 2192.08 sec
Validation loss: 0.1210, Validation accuracy: 0.9616
Epoch 60: progress time is 2629.62 sec
Validation loss: 0.1107, Validation accuracy: 0.9632
Epoch 70: progress time is 3067.43 sec
Validation loss: 0.0554, Validation accuracy: 0.9796
Epoc

## cifar100

In [8]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
최초 node의 갯수 : (tensor(1792., device='cuda:1'), tensor(1792., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:1'), tensor(1564., device='cuda:1'))
Epoch 10: progress time is 472.39 sec
Validation loss: 0.2312, Validation accuracy: 0.9408
Epoch 20: progress time is 984.71 sec
Validation loss: 0.2370, Validation accuracy: 0.9356
Epoch 30: progress time is 1497.91 sec
Validation loss: 0.3623, Validation accuracy: 0.9076
Epoch 40: progress time is 2009.89 sec
Validation loss: 0.3709, Validation accuracy: 0.8976
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:1'), tensor(1368., device='cuda:1'))
Epoch 50: progress time is 2523.62 sec
Validation loss: 0.7148, Validation accuracy: 0.8064
Epoch 60: progress time is 3036.28 sec
Validation loss: 0.7060, Validation accuracy: 0.8120
Epoch 70: progress time is 3549.32 sec
Validation loss: 0.5686, Validation accuracy: 0.8448
Epo

# Cosine scheduler

In [9]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

# cosine

## cifar10

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_cosine_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
최초 node의 갯수 : (tensor(1792., device='cuda:1'), tensor(1792., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:1'), tensor(1564., device='cuda:1'))
Epoch 10: progress time is 573.64 sec
Validation loss: 0.0213, Validation accuracy: 0.9944
Epoch 20: progress time is 1014.08 sec
Validation loss: 0.0189, Validation accuracy: 0.9952
Epoch 30: progress time is 1453.07 sec
Validation loss: 0.0172, Validation accuracy: 0.9960
Epoch 40: progress time is 1890.88 sec
Validation loss: 0.0145, Validation accuracy: 0.9952
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:1'), tensor(1368., device='cuda:1'))
Epoch 50: progress time is 2344.71 sec
Validation loss: 0.0355, Validation accuracy: 0.9880
Epoch 60: progress time is 2807.72 sec
Validation loss: 0.0345, Validation accuracy: 0.9896
Epoch 70: progress time is 3268.79 sec
Validation loss: 0.0263, Validation accuracy: 0.9924
Ep

## cifar100

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_cosine_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
최초 node의 갯수 : (tensor(1792., device='cuda:1'), tensor(1792., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:1'), tensor(1564., device='cuda:1'))
Epoch 10: progress time is 439.93 sec
Validation loss: 0.1790, Validation accuracy: 0.9584
Epoch 20: progress time is 877.5 sec
Validation loss: 0.2038, Validation accuracy: 0.9536
Epoch 30: progress time is 1315.64 sec
Validation loss: 0.2413, Validation accuracy: 0.9356
Epoch 40: progress time is 1754.34 sec
Validation loss: 0.2016, Validation accuracy: 0.9512
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:1'), tensor(1368., device='cuda:1'))
Epoch 50: progress time is 2191.94 sec
Validation loss: 0.4702, Validation accuracy: 0.8684
Epoch 60: progress time is 2628.81 sec
Validation loss: 0.4348, Validation accuracy: 0.8888
Epoch 70: progress time is 3066.55 sec
Validation loss: 0.4121, Validation accuracy: 0.8904
Epoc

## test 결과 

### cifar10

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
Total examples is 10000, correct examples is 9485; Test accuracy: 0.9485
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 9486; Test accuracy: 0.9486
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 9444; Test accuracy: 0.9444
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 9538; Test accuracy: 0.9538


### cifar100

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x4
Total examples is 10000, correct examples is 7503; Test accuracy: 0.7503
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 7599; Test accuracy: 0.7599
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 7459; Test accuracy: 0.7459
| Wide-Resnet 28x4
Total examples is 10000, correct examples is 7565; Test accuracy: 0.7565


: 

In [8]:
# 모델 지정 및 GPU 지정

model = Wide_ResNet(28, 4, 0.3, 100).to(device)
check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x4_cifar100.pth")
model.load_state_dict(check_prune['state_dict'])
# test_model(model)

| Wide-Resnet 28x4


<All keys matched successfully>

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x4_CIFAR10_scheduler_cosine_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:0'), tensor(1792., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:0'), tensor(1564., device='cuda:0'))
Epoch 10: progress time is 525.26 sec
Validation loss: 0.2514, Validation accuracy: 0.9228
Epoch 20: progress time is 1056.95 sec
Validation loss: 0.2607, Validation accuracy: 0.9232
Epoch 30: progress time is 1582.49 sec
Validation loss: 0.2636, Validation accuracy: 0.9204
Epoch 40: progress time is 2109.66 sec
Validation loss: 0.2529, Validation accuracy: 0.9264
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:0'), tensor(1368., device='cuda:0'))
Epoch 50: progress time is 2635.22 sec
Validation loss: 0.3050, Validation accuracy: 0.9180
Epoch 60: progress time is 3160.89 sec
Validation loss: 0.1978, Validation accuracy: 0.9416
Epoch 70: progress time is 3685.1 sec
Validation loss: 0.1941, Validation accuracy: 0.9464
Epoch 80: progress t

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:0'), tensor(1792., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:0'), tensor(1564., device='cuda:0'))
Epoch 10: progress time is 525.15 sec
Validation loss: 1.0208, Validation accuracy: 0.7500
Epoch 20: progress time is 1059.74 sec
Validation loss: 0.9838, Validation accuracy: 0.7532
Epoch 30: progress time is 1586.18 sec
Validation loss: 1.0052, Validation accuracy: 0.7516
Epoch 40: progress time is 2108.99 sec
Validation loss: 0.9940, Validation accuracy: 0.7628
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:0'), tensor(1368., device='cuda:0'))
Epoch 50: progress time is 2635.18 sec
Validation loss: 1.0914, Validation accuracy: 0.7324
Epoch 60: progress time is 3158.37 sec
Validation loss: 1.0551, Validation accuracy: 0.7436
Epoch 70: progress time is 3683.51 sec
Validation loss: 1.0365, Validation accuracy: 0.7560
Epoch 80: progress ti

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:0'), tensor(1792., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:0'), tensor(1564., device='cuda:0'))
Epoch 10: progress time is 528.57 sec
Validation loss: 1.4541, Validation accuracy: 0.6428
Epoch 20: progress time is 1056.03 sec
Validation loss: 1.2486, Validation accuracy: 0.6836
Epoch 30: progress time is 1578.04 sec
Validation loss: 1.2154, Validation accuracy: 0.7000
Epoch 40: progress time is 2102.44 sec
Validation loss: 1.1359, Validation accuracy: 0.7128
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:0'), tensor(1368., device='cuda:0'))
Epoch 50: progress time is 2624.87 sec
Validation loss: 1.2228, Validation accuracy: 0.7076
Epoch 60: progress time is 3148.45 sec
Validation loss: 1.0942, Validation accuracy: 0.7300
Epoch 70: progress time is 3671.2 sec
Validation loss: 1.0273, Validation accuracy: 0.7540
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

## SEED 10

In [9]:
##scheduler 실행 : cosineannealingLR, lr 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:3'), tensor(1792., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:3'), tensor(1564., device='cuda:3'))
Epoch 10: progress time is 507.57 sec
Validation loss: 0.0414, Validation accuracy: 0.9888
Epoch 20: progress time is 1015.03 sec
Validation loss: 0.0206, Validation accuracy: 0.9948
Epoch 30: progress time is 1523.59 sec
Validation loss: 0.0227, Validation accuracy: 0.9940
Epoch 40: progress time is 2031.62 sec
Validation loss: 0.0141, Validation accuracy: 0.9952
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:3'), tensor(1368., device='cuda:3'))
Epoch 50: progress time is 2537.41 sec
Validation loss: 0.0325, Validation accuracy: 0.9904
Epoch 60: progress time is 3043.25 sec
Validation loss: 0.0383, Validation accuracy: 0.9872
Epoch 70: progress time is 3550.57 sec
Validation loss: 0.0351, Validation accuracy: 0.9876
Epoch 80: progress ti

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [13]:
##scheduler 실행 : cosineannealingLR, lr 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:3'), tensor(1792., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:3'), tensor(1564., device='cuda:3'))
Epoch 10: progress time is 507.88 sec
Validation loss: 0.1856, Validation accuracy: 0.9400
Epoch 20: progress time is 1015.84 sec
Validation loss: 0.1868, Validation accuracy: 0.9388
Epoch 30: progress time is 1523.37 sec
Validation loss: 0.1880, Validation accuracy: 0.9432
Epoch 40: progress time is 2032.32 sec
Validation loss: 0.1787, Validation accuracy: 0.9476
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:3'), tensor(1368., device='cuda:3'))
Epoch 50: progress time is 2539.09 sec
Validation loss: 0.2080, Validation accuracy: 0.9356
Epoch 60: progress time is 3054.68 sec
Validation loss: 0.1639, Validation accuracy: 0.9528
Epoch 70: progress time is 3562.13 sec
Validation loss: 0.1256, Validation accuracy: 0.9680
Epoch 80: progress ti

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##cifar100
##scheduler 실행 : cosineannealingLR, lr 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:2'), tensor(1792., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:2'), tensor(1564., device='cuda:2'))
Epoch 10: progress time is 466.39 sec
Validation loss: 0.1935, Validation accuracy: 0.9516
Epoch 20: progress time is 938.57 sec
Validation loss: 0.2254, Validation accuracy: 0.9352
Epoch 30: progress time is 1410.44 sec
Validation loss: 0.2530, Validation accuracy: 0.9332
Epoch 40: progress time is 1882.41 sec
Validation loss: 0.2221, Validation accuracy: 0.9456
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:2'), tensor(1368., device='cuda:2'))
Epoch 50: progress time is 2354.45 sec
Validation loss: 0.4396, Validation accuracy: 0.8820
Epoch 60: progress time is 2826.86 sec
Validation loss: 0.4292, Validation accuracy: 0.8860
Epoch 70: progress time is 3299.02 sec
Validation loss: 0.3907, Validation accuracy: 0.8972
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

## 결과 확인

In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x4_CIFAR10_scheduler_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x4_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9508; Test accuracy: 0.9508


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x4_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x4_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7558; Test accuracy: 0.7558


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x4_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x4_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7596; Test accuracy: 0.7596


## SEED 10

In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9482; Test accuracy: 0.9482


In [14]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9514; Test accuracy: 0.9514


### cifar100

In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x4_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7469; Test accuracy: 0.7469


: 